In [1]:
%pip install peft wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
import torch
from peft import LoraConfig, get_peft_model
import numpy as np
import os
from kaggle_secrets import UserSecretsClient
import wandb
from huggingface_hub import login

2025-05-16 09:07:56.456124: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747386476.690375      73 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747386476.756835      73 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
user_secrets = UserSecretsClient()
os.environ['hf_token'] = user_secrets.get_secret("HF_TOKEN")
os.environ['WANDB_API_KEY'] = user_secrets.get_secret("WANDB_TOKEN")
assert(os.getenv('hf_token'))
assert(os.getenv('WANDB_API_KEY'))

In [4]:
from datetime import datetime
import pytz

now_utc = datetime.now(pytz.utc)
now_colombo = now_utc.astimezone(pytz.timezone('Asia/Colombo'))
time_str = now_colombo.strftime('%Y-%b-%d--%H-%M-%S')
run_name = f'lora-{time_str}'
print(run_name)

lora-2025-May-16--14-38-14


In [5]:
wandb.login(key=os.getenv('WANDB_API_KEY'))
wandb.init(project="choreo-doc-ast-ft-lora", name=run_name)

login(token=os.getenv('hf_token'))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rtweera (rtw-rtweera) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [6]:
# Load tokenizer and model
model_id = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # Use mixed precision for efficiency
    device_map="auto"            # Automatically choose best device setup (NOTE: remove if causes problems)
)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [7]:
# Configure model for LoRA fine-tuning (Parameter-Efficient Fine-Tuning)
lora_config = LoraConfig(
    r=8,                          # Rank
    lora_alpha=16,                # Alpha parameter for LoRA scaling
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Attention layers to fine-tune
    lora_dropout=0.10,
    bias="none",
    task_type="CAUSAL_LM"
)

In [8]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Shows percentage of parameters being trained

trainable params: 1,081,344 || all params: 495,114,112 || trainable%: 0.2184


In [9]:
# Load dataset
dataset = load_dataset("json", data_files="/kaggle/input/choreo-dataset/choreo_dataset.jsonl")

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
# Proper formatting for Qwen models' chat template based on Ollama template
# def format_chat(instruction, input_text, output):
#     # Using Qwen's chat template structure from Ollama
#     if instruction:
#         # Use instruction as system message
#         formatted = f"<|im_start|>system\n{instruction}<|im_end|>\n"
#     else:
#         formatted = ""
    
#     # Add user input (if any)
#     if input_text:
#         formatted += f"<|im_start|>user\n{input_text}<|im_end|>\n"
    
#     # Add assistant response
#     formatted += f"<|im_start|>assistant\n{output}<|im_end|>"
    
#     return formatted

In [11]:
# Split the dataset into training and validation sets
dataset = dataset["train"].train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

In [12]:
# Preprocess and tokenize dataset
def preprocess_function(examples):
    # We are providing same instruction for every message, so tokenize only once (for optimization)
    system_instruction = examples['instruction'][0] # select 1st instruction

    # format each example
    formatted_texts = []
    for inp, out in zip(examples['input'], examples['output']):
        # Using consistent system message with varied user inputs and assistant outputs
        formatted = f"<|im_start|>system\n{system_instruction}<|im_end|>\n<|im_start|>user\n{inp}<|im_end|>\n<|im_start|>assistant\n{out}<|im_end|>"
        formatted_texts.append(formatted)
    
    tokenized = tokenizer(
        formatted_texts,
        padding="max_length",
        truncation=True,
        max_length=1024,
        return_tensors="pt"
    )
    
    # Create labels (for causal LM, typically identical to input_ids)
    tokenized["labels"] = tokenized["input_ids"].clone()
    
    return tokenized

In [13]:
# Apply preprocessing to datasets
tokenized_train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_eval_dataset = eval_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=eval_dataset.column_names
)

Map:   0%|          | 0/607 [00:00<?, ? examples/s]

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

In [14]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./qwen_choreo_ft_lora",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=4,
    eval_accumulation_steps=4,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_steps=10,
    logging_first_step=True,
    eval_on_start=True,
    eval_strategy="steps",
    eval_steps=20,
    save_strategy="steps",
    fp16=True if torch.cuda.is_available() else False,
    logging_dir="./logs",
    load_best_model_at_end=True,
    save_total_limit=3,
    report_to="wandb",
    push_to_hub=True,
    run_name=run_name
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
)

/tmp/ipykernel_73/3871543629.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [15]:
# Start training
trainer.train()

Step,Training Loss,Validation Loss
0,No log,10.897990
20,9.651800,9.337799
40,7.474400,7.225341
60,4.928900,4.423046
80,2.268400,1.816601
100,1.155600,1.017570
120,0.755300,0.767846
140,0.741400,0.734617
160,0.734700,0.709032
180,0.662500,0.685328


TrainOutput(global_step=760, training_loss=1.290613853931427, metrics={'train_runtime': 3232.3566, 'train_samples_per_second': 3.756, 'train_steps_per_second': 0.235, 'total_flos': 2.677561710084096e+16, 'train_loss': 1.290613853931427, 'epoch': 20.0})

In [16]:
# Save the model
model.save_pretrained("./qwen_choreo_ft_final")
tokenizer.save_pretrained("./qwen_choreo_ft_final")

('./qwen_choreo_ft_final/tokenizer_config.json',
 './qwen_choreo_ft_final/special_tokens_map.json',
 './qwen_choreo_ft_final/vocab.json',
 './qwen_choreo_ft_final/merges.txt',
 './qwen_choreo_ft_final/added_tokens.json',
 './qwen_choreo_ft_final/tokenizer.json')

In [27]:
# Optionally, test the model with a sample
test_input = "How do I configure my applications to scale automatically?"
formatted_test = f"<|im_start|>user\n{test_input}<|im_end|>\n<|im_start|>assistant\n"
inputs = tokenizer(formatted_test, return_tensors="pt").to(model.device)

# Generate response
outputs = model.generate(
    input_ids=inputs["input_ids"],
    max_new_tokens=1000,
    temperature=0.5,
    top_p=0.9,
    do_sample=True
)

# Decode and print response
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<|im_start|>user
How do I configure my applications to scale automatically?<|im_end|>
<|im_start|>assistant
To configure your applications to scale automatically, you can follow these steps:

1. **Understand the Scaling Requirements:**
   - Determine how many instances of each application are needed based on the expected traffic and business requirements.
   - Consider factors such as CPU, memory, storage, network bandwidth, and other resources required by your applications.

2. **Configure the Application Autoscaling Settings:**
   - In most environments, you need to set up autoscaling for your applications. This involves defining a scaling policy that will trigger when the application's workload exceeds its capacity.
   
3. **Set Up the Autoscaler:**
   - Go to the Kubernetes dashboard or the Kubernetes Console.
   - Navigate to the namespace where your applications reside.
   - Click on `Autoscaling` in the left sidebar.
   - Choose the appropriate type of autoscaling (e.g., Horizon